In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pretty_midi as pm
import numpy as np
import io
import matplotlib.pyplot as plt
import sys
import pygame

2023-04-14 00:37:51.718194: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/benmodlin/miniconda3/envs/py4sci/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


pygame 2.3.0 (SDL 2.24.2, Python 3.10.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# tfds works in both Eager and Graph modes
tf.executing_eagerly()

True

In [3]:
# Load the full GMD with MIDI only (no audio) as a tf.data.Dataset
dataset = tfds.load(
    name="groove/full-midionly",
    split=tfds.Split.TRAIN,
    try_gcs=True)

2023-04-14 00:39:58.644987: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".
2023-04-14 00:40:02.064810: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Build your input pipeline
dataset = dataset.shuffle(1024).batch(64).prefetch(
    tf.data.experimental.AUTOTUNE)

In [5]:

for features in dataset.take(1):
  # Access the features you are interested in
  midi, genre = features["midi"], features["style"]["primary"]

In [6]:
print(midi[0])


tf.Tensor(b'MThd\x00\x00\x00\x06\x00\x00\x00\x01\x01\xe0MTrk\x00\x00\x01\x04\x00\xff\x03\nMidi Drums\x00\xff\x04\x08Brooklyn\x00\xffX\x04\x04\x02\x18\x08\x00\xffY\x02\x00\x00\x00\xffT\x05!\x00\x00\x00\x00\x00\xffQ\x03\x0fB@\x01\xb9\x04\x19\x1c\x04\x1b<\x04\x1d2\x04\x1f\x1e\x04 \x1e\x04"\x02\x04#\x1e\x04&\x08\x04(<\x04&\x1e\x04$\x1e\x04\x1f\x13\x04\x1b\x1e\x04\x18\x05\x04\x17\x0f\x04$\x0f\x041\x0f\x04>\x0f\x04K\x02\x99,\x11\r\xb9\x04X\x02\x04Z!\x89,@\x1b\xb9\x04Y<\x04X<\x04W<\x04V!\x04U\x0f\x04Q\x0f\x04L\x01\x04K\x0f\x04G\x0f\x04B\x02\x04A\x0f\x048\x0f\x04.\x0b\x04&0\x04(\x0f\x04>\x0f\x04T\x04\x04Z|\x99&H0\x89&@\x16\xb9\x04Y:\x99&S1\x89&@A\x990O\x16\xb9\x04X\x1a\x890@;\x990\\1\x890@<\xb9\x04W\t\x99-Q0\x89-@N\x99+D0\x89+@\x0b\xb9\x04V5\x99+@0\x89+@\x00\xff/\x00', shape=(), dtype=string)


In [15]:
def play_midi(midi_tensor):
    midi_bytes = midi_tensor.numpy()
    midi_file = io.BytesIO(midi_bytes)

    pygame.init()
    pygame.mixer.music.load(midi_file)
    pygame.mixer.music.play()

    while pygame.mixer.music.get_busy():
        pass

    pygame.quit()

In [18]:
for i in range(3):
    play_midi(midi[i])

In [21]:
def convert_to_sequence(midi_tensor):
    midi_bytes = midi_tensor.numpy()
    midi_file = io.BytesIO(midi_bytes)
    midi_data = pm.PrettyMIDI(midi_file)
    
    time_resolution = midi_data.tick_to_time(1)
    duration = midi_data.get_end_time()
    total_ticks = int(duration / time_resolution) + 1
    
    note_sequence = np.zeros((total_ticks, 128))
    
    for instrument in midi_data.instruments:
        for note in instrument.notes:
            tick_start = int(note.start / time_resolution)
            tick_end = int(note.end / time_resolution)
            
            note_sequence[tick_start:tick_end, note.pitch] = 1
    
    return note_sequence

In [22]:
roll = convert_to_sequence(midi[0])
print(roll.shape)

44
38
38
48
48
45
43
43
(1827, 128)


In [ ]:
np.set_printoptions(threshold=sys.maxsize)

print(roll)